# FILTROS II

## Índice

[Enunciado](#Enunciado)

[Apartado a](#Apartado-a)

[Apartado b](#Apartado-b)

[Apartado c](#Apartado-c)

[Apartado d](#Apartado-d)

## Enunciado

**a.** Comprueba la propiedad de "cascading" del filtro gaussiano.

**b.** Comprueba la propiedad de "separabilidad" del filtro gaussiano. 

**c.** Implementa en Python dede cero (usando bucles) el algoritmo de convolución con una máscara general y compara su eficiencia con la versión de OpenCV. 

**c.** Impleméntalo en C y haz un "wrapper" para utilizarlo desde Python (consulta al profesor). 

**d)** Implementa el box filter con la imagen integral.

In [ ]:
# Librerías usadas
import numpy             as np
import cv2               as cv
import matplotlib.pyplot as plt
import scipy.signal      as signal
import time

## Apartado a

La propiedad cascading se refiere a que, a medida que se aplican filtros gaussianos a una imagen de forma consecutiva, esta cada vez se va suavizando más. Se procede a mostrar su funcionamiento mediante un código en Python que aplica el filtro gaussiano (con el mismo kernel) a una imagen de forma consecutiva:

In [ ]:
import math

# Se crea la máscara de filtro gaussiano
mascara = np.array([[1,2,1],[2,4,2],[1,2,1]])
mascara = mascara/np.sum(mascara)

# Se lee la imagen y se pasa a escala de grises
imagen = cv.imread('./foto.png')
imagen = cv.cvtColor(imagen, cv.COLOR_BGR2GRAY)

# Bucle para mostrar las imágenes a las que se le aplican 0..(n-1) filtros gaussianos
imagenplot = imagen
n = 9
num_filas = math.ceil(math.sqrt(n))
num_columnas = math.ceil(n / num_filas)
plt.figure(figsize=(num_columnas * 6, num_filas * 4))

for i in range(n):
    plt.subplot(num_filas, num_columnas, i + 1)
    plt.imshow(imagenplot, cmap='gray')
    if i == 0:
        plt.title("Imagen sin filtros")
    else:
        plt.title(f'Imagen aplicados {i} filtros gaussianos')
    plt.axis('off')
    imagenplot = cv.filter2D(imagenplot,-1,mascara)
plt.show()


Como se puede observar, cuando se van aplicando filtros, se va suavizando la imagen. Esto se corresponde con lo dicho con anterioridad.

Se puede modificar el número de imágenes mostradas modificando el valor de n.

## Apartado b

El filtro gaussiano sirve para suavizar una imagen. Este se puede aplicar mediante una máscara de 3x3, de forma que se suaviza tomando en cuenta los vecinos del píxel.

Sin embargo, este también se puede aplicar mediante dos máscaras, una de 3x1 y otra de 1x3, de forma que en primer lugar los píxeles se mezclan con sus vecinos en vertical, y después su resultado mezcla sus vecinos en horizontal (o viceversa). Esto es la propiedad de separabilidad, las máscaras en 2 dimensiones aplicadas a una imagen también se pueden aplicar usando dos máscaras de 1 dimensión a la imagen de forma consecutiva. 

Se muestra de forma gráfica para que se entienda mejor. La siguiente máscara es una máscara 3x3 que aplica el filtro gaussiano (se multiplica por 1/16 porque la suma de los valores de la máscara es $1*4+2*4+4=16$):

\begin{equation}
\frac{1}{16}*\begin{pmatrix}
1 & 2 & 1\\
2 & 4 & 2\\
1 & 2 & 1
\end{pmatrix} = \begin{pmatrix}
\frac{1}{16} & \frac{1}{8} & \frac{1}{16}\\
\frac{1}{8} & \frac{1}{4} & \frac{1}{8}\\
\frac{1}{16} & \frac{1}{8} & \frac{1}{16}
\end{pmatrix}
\end{equation}

Si se aplican las siguientes máscaras sobre la imagen, se genera el mismo resultado (se multiplica por $1/4$ por la misma razón que antes se multiplicaba por $1/16$, y es que los valores de las máscaras son $1*2+2=4$):

\begin{equation}
\frac{1}{4}*\begin{pmatrix}
1\\
2\\
1
\end{pmatrix} = \begin{pmatrix}
\frac{1}{4}\\
\frac{1}{2}\\
\frac{1}{4}
\end{pmatrix}\end{equation}

\begin{equation}
\frac{1}{4}*\begin{pmatrix}
1 & 2 & 1
\end{pmatrix} = \begin{pmatrix}
\frac{1}{4} & \frac{1}{2} & \frac{1}{4}
\end{pmatrix}
\end{equation}

Esto ocurre porque \begin{equation} \begin{pmatrix}
\frac{1}{4}\\
\frac{1}{2}\\
\frac{1}{4}
\end{pmatrix} * \begin{pmatrix}
\frac{1}{4} & \frac{1}{2} & \frac{1}{4}
\end{pmatrix} = \begin{pmatrix}
\frac{1}{16} & \frac{1}{8} & \frac{1}{16}\\
\frac{1}{8} & \frac{1}{4} & \frac{1}{8}\\
\frac{1}{16} & \frac{1}{8} & \frac{1}{16}
\end{pmatrix}
\end{equation}

Se aplica el filtro con la máscara de 3x3 y con las otras máscaras a una imagen, y se comprueba si da lo mismo:

In [ ]:
# Se lee la imagen y se pasa a escala de grises
imagen = cv.imread('./foto.png')
imagen = cv.cvtColor(imagen, cv.COLOR_BGR2GRAY)

# Se aplica la máscara 3x3
mascara3x3 = np.array([[1,2,1],[2,4,2],[1,2,1]])
mascara3x3 = mascara3x3/np.sum(mascara3x3)
imagen_3x3 = cv.filter2D(imagen,-1,mascara3x3)

# Se aplican las máscaras 3x1 y 1x3
mascara1 = np.array([[1],[2],[1]])
mascara1 = mascara1/np.sum(mascara1)
mascara2 = np.array([1,2,1])
mascara2 = mascara2/np.sum(mascara2)
imagen_2_mascaras = cv.filter2D(cv.filter2D(imagen, -1, mascara1),-1,mascara2)

# Se mira la máxima diferencia:
iguales = np.equal(imagen_2_mascaras,imagen_3x3)
imagen_h, imagen_w = imagen.shape
maxima_diferencia = 0
for i in range(0,imagen_h-1):
    for j in range(0,imagen_w-1):
        if (iguales[i,j] == False):
            diferencia = abs(int(imagen_2_mascaras[i,j]) - int(imagen_3x3[i,j]))
            if diferencia < 0: 
                diferencia = - diferencia
            if diferencia > maxima_diferencia: 
                maxima_diferencia = diferencia
                #print("No son iguales, tienen diferencia de ", diferencia, " se diferencian en i =", i, " j = ", j, "por que tienen valores ", imagen_2_mascaras[i,j], " y ", imagen_3x3[i,j])
print("La máxima diferencia es:", maxima_diferencia)

In [ ]:
# Se dibujan las imágenes obtenidas
plt.figure(figsize=(12,4))
plt.subplot(1,2,1); plt.imshow(imagen_3x3, cmap='gray'); plt.axis("off"); plt.title("3x3");
plt.subplot(1,2,2); plt.imshow(imagen_2_mascaras, cmap='gray'); plt.axis("off"); plt.title("2 máscaras")

Como se ha podido observar, la propiedad de separabilidad no se ha podido demostrar ya que las imágenes una vez aplicados los filtros no eran iguales.

## Apartado c

En primer lugar, se implementa el algoritmo de convolución con una máscara general (array de numpy):

In [ ]:
def conv(img, array):
    H,W = img.shape
    resultado = img.copy()
    array_h, array_w = array.shape
    array_mitad_h = (array_h-1)//2
    array_mitad_w = (array_w-1)//2
    
    # Recorre todos los píxeles de la imagen
    for h in range(0,H-1):
        for w in range(0,W-1):
            # Por cada píxel
            pixel_resultante = 0
            # Recorre los vecinos del píxel (array)
            for vecino_h in range(-array_mitad_h,array_mitad_h+1): 
                for vecino_w in range(-array_mitad_w,array_mitad_w+1):
                    # Posición del vecino en la imagen: [i, j]
                    i = vecino_h+h 
                    j = vecino_w+w
                    # Si se ha salido de la imagen, se coge el píxel del borde de la imagen
                    if i < 0:
                        i = 0
                    elif i >= H:
                        i = H-1
                    if j < 0:
                        j = 0
                    elif j >= W:
                        j = W-1
                    # pixel_resultante += El valor del píxel vecino en la imagen por el valor que tiene ese vecino en la máscara
                    pixel_resultante +=  img[i,j]*array[vecino_h+array_mitad_h][vecino_w+array_mitad_w]
            resultado[h,w] = pixel_resultante
    return resultado


Se ha realizado el filtro dando por hecho que la imagen sólo tiene un valor por píxel (blanco y negro) para que la multiplicación de cada píxel por el valor que corresponda de la máscara se realice de forma sencilla. A su vez, también se ha dado por hecho que las filas y columnas del array de entrada (máscara) son impares.

Se realiza un bucle por todos los píxeles de la imagen, y se le asigna a cada píxel la suma de los valores de los vecinos (y del píxel en sí) por el valor de la posición de cada uno en el array de la máscara. Si un vecino no existe (píxel en el borde de la imagen), se coje el píxel más cercano a este.

Una vez implementado, se ejecuta con una imagen y una máscara. Se presenta la imagen original a la izquierda para que se vea el efecto del filtro.

Se ha utilizado un filtro de los [apuntes de la asignatura](https://github.com/albertoruiz/umucv) ya que se ha considerado interesante el efecto de este, al verse de forma rápida si el filtro realiza lo que debería.

In [ ]:
# Se lee la imagen y se pasa a escala de grises
imagen = cv.imread('./foto.png')
imagen = cv.cvtColor(imagen, cv.COLOR_BGR2GRAY)

# Se crea la máscara (ker) sacada de los apuntes de la asignatura
ker = np.zeros([11,11])
ker[0,0] = 1
ker[10,10] = 1
ker = ker/np.sum(ker)

# Se dibujan la imagen sin modificar en blanco y negro (izquierda), y la imagen pasada por el algoritmo de convolución (derecha)
plt.figure(figsize=(12,4))
plt.subplot(1,2,1); plt.imshow(imagen, cmap='gray'); plt.axis("off");
plt.subplot(1,2,2); plt.imshow(conv(imagen, ker), cmap='gray'); plt.axis("off")

Se calcula el tiempo que tarda en ejecutarse el algoritmo de convolución con la imagen y la máscara anteriores, tanto con el algoritmo creado (conv) como con el ofrecido por OpenCV:

In [ ]:
inicio = time.time()
conv(imagen, ker)
fin = time.time()
print("Tiempo del algoritmo implementado = ", fin-inicio)

inicio = time.time()
cv.filter2D(imagen,-1,ker)
fin = time.time()
print("Tiempo del algoritmo de OpenCV = ", fin-inicio)

Considerando que el tiempo de ejecución puede variar según múltiples factores como el hardware, indico los tiempos que me indicó al ejecutar la versión implementada por mí y la de OpenCV para poder comentarlo.

Mi implementación tardó 35.3 segundos, en contraste de la versión de OpenCV que indicó 0.0.

Esto proporciona una idea de lo importante que son las librerías de código en la programación, dado que estas se encuentran muy optimizadas (sobre todo las populares).

# Apartado d

Para este apartado, me he basado en la explicación del siguiente vídeo de youtube https://www.youtube.com/watch?v=5ceT8O3k6os, que explica qué son las imágenes integrales de forma muy clara y concisa (5 minutos, 28 segundos).

Las **imágenes integrales** se calculan a partir de las imágenes. Cada píxel de la imagen integral tiene como valor la suma de todos los valores de los píxeles superiores (y misma fila) y a la derecha (y misma columna) de este píxel en la imagen original.

De esta forma, se procede a hacer un método que calcule la imagen integral a partir de la imagen original (como en casos anteriores, se supone una imagen en escala de grises).

In [ ]:
def integral(imagen):
    integral = np.zeros_like(imagen, np.int64)
    H, W = imagen.shape
    # h=0
    integral[0,0] = imagen[0,0]
    for w in range(1,W):
        integral[0,w] = integral[0,w-1] + imagen[0,w]
    # w=0
    for h in range(1,H):
        integral[h,0] = integral[h-1,0] + imagen[h,0]
    # h!=0 w!=0
    for h in range(1,H):
        for w in range(1, W):
            integral[h,w] = integral[h-1,w] + integral[h,w-1] - integral[h-1, w-1] + imagen[h,w]
    return integral

Se ha seguido un método donde primero se calculan los píxeles con h=0 y con w=0, ya que estos son la suma de ese píxel en la imagen original más el anterior ya calculado en la fila/columna. Se enseña una imagen para mejor comprensión:

<img src="./imagen-integral-1.png" style="width:25%"></img>

En la imagen integral, el valor del píxel 1 es el píxel 1 en la imagen original, el píxel 2 es la suma del 1 y 2 de la imagen original (píxel 2 en la original y 1 del integral), el píxel 3 es la suma del 1, 2 y 3 (es decir, la suma del píxel 2 en la integral y del 1 en la original), ...

A su vez, el valor del píxel 8 en la integral es el valor del pixel 1 y el 8 en la original, el valor del píxel 9 en la integral es el valor del píxel 9 en la original y del 8 en la integral, ...

Por esta razón se ha usado dos bucles para calcular estas primera fila/columna, sumando el píxel de la original y el anterior de la integral.

Posteriormente, se ha pasado al resto de la imagen. Para calcular estos píxeles se usa el valor de los píxeles de el de arriba, el de la izquierda, y el de arriba a la izquierda. Como se hace después de calcular la primera fila y columna, estos píxeles siempre van a existir en la imagen.

Se hace un dibujo para que se entienda mejor:

<img src="./imagen-integral-2.png" style="width:25%"></img>

El valor del píxel amarillo en la imagen integral es la suma de todos los valores de los píxeles coloreados que aparecen, en la imagen original. 

Para hacer esto, en primer lugar se hace un bucle de arriba a abajo y de izquierda a derecha, de modo que se tengan los píxeles de arriba y de la izquierda ya calculados en la imagen integral para calcular el siguiente píxel de esta imagen integral.

Dentro del bucle, para cada píxel, se hace la suma del píxel de encima en la imagen integral (es la suma de los valores de los píxeles morados) y del píxel de la izquierda en la imagen integral (es la suma de los valores de los píxeles azules). Esto genera que los píxeles tanto morados como azules se hayan sumados dos veces, por lo que se resta el píxel de arriba a la izquierda en la imagen integral (es la suma de los píxeles en naranja, que como se ve también son los píxeles azules y morados que se han sumados dos veces).

Una vez obtenida la imagen integral, se debe de hacer el **filtro box** con la imagen integral. El filtro box puede tener un número variables de filas y columnas, por lo que se va a implementar de forma que se puedan indicar las filas y columnas de la imagen (deben ser ambas impares).

In [ ]:
# Se pasan como parámetro las filas, columnas, y la imagen integral
def filtroBox(imagen, filas, columnas):
    # Si el número de filas o columnas es par, devolver la imagen tal cual e indicar que el número de filas/columnas es par y esto no se puede
    if filas%2 == 0 or columnas%2 == 0:
        print("filas%2 == 0 or columnas%2 == 0")
        return imagen
    imagenBox = np.zeros_like(imagen, np.uint8)
    H, W = imagen.shape
    for h in range(0, H):
        for w in range(0,W):
            fila_inicio = h+filas//2 # x + 1//2 = x
            columna_inicio = w+columnas//2 # 506 + 1//2 = 506
            fila_fin = fila_inicio - filas # x - 1
            columna_fin = columna_inicio - columnas # 506 - 1 = 505
            # Si la fila o columna del inicio están fuera de la imagen
            if fila_inicio >= H:
                fila_inicio = H-1
            if columna_inicio >= W:
                columna_inicio = W-1
            # Si la fila o columna del fin están fuera de la imagen
            if fila_fin < 0:
                if columna_fin < 0:
                    imagenBox[h,w] = imagen[fila_inicio, columna_inicio]
                    imagenBox[h,w] = imagenBox[h,w]/((fila_inicio+1)*(columna_inicio+1))
                else:
                    imagenBox[h,w] = imagen[fila_inicio, columna_inicio] - imagen[fila_inicio, columna_fin]
                    imagenBox[h,w] = imagenBox[h,w]/((fila_inicio+1)*(columna_inicio-columna_fin))
            elif columna_fin < 0:
                imagenBox[h,w] = imagen[fila_inicio, columna_inicio] - imagen[fila_fin, columna_inicio]
                imagenBox[h,w] = imagenBox[h,w]/((fila_inicio-fila_fin)*(columna_inicio+1))
            else:
                imagenBox[h,w] = ((imagen[fila_inicio, columna_inicio] - imagen[fila_fin, columna_inicio] - imagen[fila_inicio, columna_fin] + imagen[fila_fin, columna_fin])/((fila_inicio-fila_fin)*(columna_inicio-columna_fin)))
    return imagenBox

Para mostrar lo que hace el código, se muestra una imagen para que se vea de forma más clara:

<img src="./imagen-integral-3.png" style="width:25%"></img>

Para conseguir la suma de los valores alrededor del píxel amarillo (3 filas y 3 columnas), se debe obtener la suma de los valores que sólo tienen color azul (incluido el amarillo).

Para ello, con la imagen integral, se accede al píxel número 1, y a su valor se le resta el valor del píxel 2 (suma de píxeles rosas) y del píxel 3 (suma de píxeles morados). De esta forma, se tiene que se han restado dos veces los valores en rojo, por lo que se vuelven a sumar estos valores (píxel 4).

De esta forma, se han obtenido la suma de valores de los píxeles que sólo son azules. Esto es lo que se realiza en el código de *filtroBox*, y sus variables son:
- *fila_inicio*: La fila de la casilla 1
- *fila_fin*: La fila de la casilla 4
- *columna_inicio*: Columna de la casilla 1
- *columna_fin*: Columna de la casilla 4

Una vez obtenidos estos valores, se calcula el valor de cada píxel mediante la suma/resta de 4 valores. Esto se puede extender a distintos valores de filas y columnas, obteniendo siempre el píxel 1 como el de la esquina derecha-abajo del retángulo requerido, el píxel 4 como la esquina arriba-izquierda de las afueras del rectángulo, y los píxeles 2 y 3 se obtienen sabiendo las filas y columnas de los píxeles 1 y 4.

Sin embargo, se tienen las siguientes excepciones:
- fila_inicio >= H: Si la fila de inicio está fuera de la imagen, se guarda como fila_inicio la última fila, de forma que se tiene un rectángulo más pequeño al estar pegado a los bordes
- columna_inicio >= W: Si la columna de inicio está fuera de la imagen, se guarda como columna_inicio la última columna, y al igual que en el anterior caso, se tiene un rectángulo más pequeño al estar pegado a los bordes
- fila_fin < 0: Si la fila de fin está fuera de la imagen, ya no se resta el píxel 2 (no hay valores que restar). Por esa misma razón, tampoco se suma el píxel 4.
- columna_fin < 0: Si la columna de fin está fuera de la imagen, ya no se resta el píxel 3 (no hay valores que restar). Por esa misma razón, tampoco se suma el píxel 4.
- fila_fin < 0 y columna_fin < 0: Simplemente se accede al valor del píxel 1

Una vez sumado, se tiene que dividir por el número de píxeles. No es filas\*columnas por haber píxeles cuyo rectángulo es más pequeño por estar en los bordes, por lo que es:
- fila: El valor de la fila depende de la fila de fin:
    - fila_fin < 0: Entonces, el tamaño en vertical del rectángulo de celdas sumadas es el valor de la fila de inicio (+1 por empezar a contar en 0)
    - fila_fin >= 0: Entonces, el tamaño es la fila de inicio menos la fila de fin
- columna: El valor de la columna depende de la columna de fin_
    - columna_fin < 0: Ocurre lo mismo que con las filas. El tamaño en horizontal del rectángulo de celdas es el valor de la columna de inicio +1 (tambien se empieza a contar en 0).
    - columna_fin >= 0: Entonces, el tamaño es la columna de inicio menos la de fin
    
Es importante recordar que la fila y columna de inicio se modificó para que estuviera dentro de la imagen.

Una vez implementado el filtro box, se **muestra la imagen original y la pasada por el filtro** box (se hace con filas=5 y columnas=5, pero se puede modificar cambiando la línea en la que se aplica el filtro *filtroBox(integralImagen, 5,5)* modificando los dos últimos valores).

In [ ]:
# Se lee la imagen y se pasa a escala de grises
imagen = cv.imread('./foto.png')
imagen = cv.cvtColor(imagen, cv.COLOR_BGR2GRAY)
# Se obtiene la imagen integral y, se usa para obtener la imagen con el filtro de Box
integralImagen = integral(imagen)
filteredImagen = filtroBox(integralImagen, 5,5)

# Se dibujan la imagen sin modificar en blanco y negro (izquierda), y la imagen con el filtro box aplicado (derecha)
plt.figure(figsize=(12,4))
plt.subplot(1,2,1); plt.imshow(imagen, cmap='gray'); plt.axis("off");
plt.subplot(1,2,2); plt.imshow(filteredImagen, cmap='gray'); plt.axis("off")

Por último, para **comprobar** que el filtro box **funciona de forma correcta**, se comprueba que la imagen original y la pasada por el filtro box con tamaño de fila y de columna 1 son iguales.

Deben ser iguales porque el filtro box aplicado a una imagen da como valor en cada píxel la media de él y sus vecinos de la imagen original, y si se escoje 1 sola fila y columna, sólo se tiene en cuenta el pixel. Por ello, los valores de cada píxel en la imagen original y en la imagen a la que se aplicó el filtro box deben ser iguales.

In [ ]:
imagen = cv.imread('./foto.png')
imagen = cv.cvtColor(imagen, cv.COLOR_BGR2GRAY)
integralImagen = integral(imagen)
filteredImagen = filtroBox(integralImagen, 1,1)
np.equal(imagen,filteredImagen).all()

*np.equal* devuelve un array de las mismas proporciones de los arrays pasados como parámetro donde si los valores correspondientes eran iguales en ambos arrays, en el devuelto hay True en esa posición, y si no, False. all sólo devuelve True si todos los valores del array son True. Si hay alguno a False, devuelve False.

Al haber devuelto True, queda demostrado que las imágenes original, y pasada por el filtro box con fila y columna tamaño 1, son iguales.

## Bibliografía

[Apuntes de la asignatura](https://github.com/albertoruiz/umucv)

[ChatGPT para saber usar math.ceil y plt.figure para hacer plots de un número de imágenes variables](https://chat.openai.com/)

[Imágenes integrales](https://www.youtube.com/watch?v=5ceT8O3k6os)

[Crear array con ceros](https://numpy.org/doc/stable/reference/generated/numpy.zeros_like.html)

[np.equal](https://numpy.org/doc/stable/reference/generated/numpy.equal.html)

[función all](https://www.geeksforgeeks.org/python-all-function/)